In [1]:
import numpy as np
from scipy.integrate import quad
from scipy import constants

Mj = 1.0
Mp = 10*Mj
orbit = 5.0 # AU

In [3]:
print constants.c / 1.e8

2.99792458


So `constants` has values in MKS.

In [10]:
def planck(wl, *args):
    """
    Returns the Planck function evaluated at wavelength wl and temperature temp.
    """
    wl = np.array(wl)
    temp, = args
    fac = constants.h * constants.c / (wl * constants.k * temp)
    return (2.0 * constants.h * constants.c**2 / (wl**5)) * (1. / (np.exp(fac) - 1.))

In [13]:
def planck_photon(wl, *args):
    """
    Returns the thermal intensity of photons at wavelength wl and temperature temp.
    """
    wl = np.array(wl)
    energy_intensity = planck(wl, *args)
    energy_per_photon = constants.h * constants.c / wl
    return energy_intensity / energy_per_photon

In [14]:
wls = [1.e-6, 2.e-6]
temp = 300.
print planck(wls, temp)
print planck_photon(wls, temp)

[  1.76806236e-07   1.43404410e+02]
[  8.90063281e+11   1.44382916e+21]


In [15]:
# check with Solar Constant
Rsun_to_AU = 0.0046
quad(planck, 0.01e-6, 5.0e-6, args=(5800.0,))[0]*Rsun_to_AU**2*np.pi

1350.778497996132

In [16]:
# accretion temperature
Tacc=1e5*(Mp/Mj) # K

# accretion luminosity
Luminosity = 1e25*(Mp/Mj)**3

# "class B" recombination rate
beta = 2.74e-14*(Mp/Mj)**(-3.0/4.0)

In [17]:
planck_integrated = quad(planck, 0, 1.0e-4, args=(Tacc))[0]*np.pi
print planck_integrated

5.67217787748e+16


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/scipy/integrate/quadpack.py:352: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  warnings.warn(msg, IntegrationWarning)


In [18]:
# denoting that surface area by "surface", 
###  surface * planck_integrated = Luminosity
###  surface =  Luminosity  / planck_integrated 

surface = Luminosity  / planck_integrated 
print surface

1.76299125592e+11


In [19]:
Luminosity / ( constants.sigma*Tacc**4 )

176355241533.4935

In [20]:
# Suppose photons with energy > 13.6 eV can ionize hydrogen

eV_to_Joule = 1.6e-19
wl_ionize = constants.h * constants.c / (13.6 * eV_to_Joule )
print wl_ionize*1e9, "[nm]"

91.2888641208 [nm]


In [21]:
# photons per second

source = surface * np.pi * quad(planck_photon, 0, wl_ionize, args=(Tacc))[0]
print source, "[s^-1]"

2.66742378549e+44 [s^-1]


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/kernel/__main__.py:8: RuntimeWarning: overflow encountered in exp


In [178]:
# DSS COMMENT: I recommend not using "magic numbers"
# https://en.wikipedia.org/wiki/Magic_number_(programming)#Unnamed_numerical_constants
# I assume the 5.0 is AU?  I don't know what the 1e3 or the 1.8e6 are, though.
# It would be easier to follow the code if you replace all magic numbers with named constants.
# You can use some judgment here, like the "**(-2)" exponent doesn't need to be named,
# but I would name nearly everything else -- even obvious things like 13.6, which could be
# named "Rydberg" or something similar.
density = 1e3 * 1.8e6 * (orbit / 5.0 )**(-2) # [cm^{-3}]

In [179]:
# DSS COMMENT: Isn't the calculation up to here in MKS, not cgs?  Do we want AU_to_m instead?
# Not sure about this one....
AU_to_cm = 1.49597870700e13
stromgren = 3.0 / ( 4.0 * np.pi ) * ( source / ( density**2 * beta ) )**(1.0/3.0) / AU_to_cm
print stromgren, "[AU]"

0.409497694315 [AU]
